In [1]:
import numpy as np
import math

### QUESTION 1 ###

# European option code from section 3.5 notes, modified into a function to return 
# present value of stock
# S0 - current stock price 
# K - strike
# T - expiry time
# r - interest rate
# sigma - volatility
# opttype - 0 for a call, otherwise a put 
# Nsteps - number of timesteps
def euoption(S0, K, T, r, sigma, opttype, Nsteps): 
    delt = T/Nsteps;
    
    # tree parameters
    u = np.exp(sigma * np.sqrt(delt) ); 
    d = 1./u;
    a = np.exp( r*delt );
    p = (a - d)/(u - d);
    
    # payoff at t=T
    W = S0 * d**(np.arange(Nsteps,-1,-1)) * u**(np.arange(Nsteps+1))
    
    # W is column vector of length Nsteps+1
    if opttype == 0:
        W = np.maximum(W - K, 0); 
    else:
        W = np.maximum(K - W, 0);
        
    # backward recursion
    for i in np.arange(Nsteps,0,-1):
        W = np.exp(-r*delt)*( p*W[1:i+1] + (1-p)*W[0:i] )
    
    return W


# Calculates the option price for American options
# S0 - current stock price 
# K - strike
# T - expiry time
# r - interest rate
# sigma - volatility
# opttype - 0 for a call, otherwise a put 
# Nsteps - number of timesteps
def amoption(S0, K, T, r, sigma, opttype, Nsteps): 
    delt = T/Nsteps;
    
    # tree parameters
    u = np.exp(sigma * np.sqrt(delt) ); 
    d = 1./u;
    a = np.exp( r*delt );
    p = (a - d)/(u - d);       
    
    # payoff at t=T
    W = S0 * d**(np.arange(Nsteps,-1,-1)) * u**(np.arange(Nsteps+1))
    
    # W is column vector of length Nsteps+1
    if opttype == 0:
        W = np.maximum(W - K, 0); 
    else:
        W = np.maximum(K - W, 0);
    
        
    # backward recursion
    for i in np.arange(Nsteps,0,-1):
        S = S0 * d**(np.arange(i-1,-1,-1)) * u**(np.arange(i));
        W = np.maximum(np.exp(-r*delt)*( p*W[1:i+1] + (1-p)*W[0:i] ), 
                       np.maximum(opttype*(K - S) + (1-opttype)*(S - K) ,0 ));
    
    return W
    




### Test data ###

teststeps = [500, 1000, 2000, 4000, 8000]

# European options
preval = [0]
prechange = 0;

print("European call options: \n");
print("Exact value from blsprice is 13.0571 \n");
for i in teststeps:
    eupricecall =  euoption(100, 100, 0.75, 0.03, 0.35, 0, i);
    print("delta = T/" + str(i));
    print("Value: " + str(eupricecall[0]));
    if (i > 500):
        print("Change: " + str(eupricecall[0] - preval[0]));
    if (i > 1000): 
        print("Ratio: " + str(prechange / (eupricecall[0] - preval[0])));
    prechange = eupricecall[0] - preval[0]
    preval = eupricecall
    print("\n");  

print("European put options: \n");
print("Exact value from blsprice is 10.8323 \n");
for i in teststeps:
    eupriceput =  euoption(100, 100, 0.75, 0.03, 0.35, 1, i);
    print("delta = T/" + str(i));
    print("Value: " + str(eupriceput[0]));
    if (i > 500):
        print("Change: " + str(eupriceput[0] - preval[0]));
    if (i > 1000): 
        print("Ratio: " + str(prechange / (eupriceput[0] - preval[0])));
    prechange = eupriceput[0] - preval[0]
    preval = eupriceput
    print("\n");
    

# American option
print("American put options: \n");
for i in teststeps:
    ampriceput =  amoption(100, 100, 0.75, 0.03, 0.35, 1, i);
    print("delta = T/" + str(i));
    print("Value: " + str(ampriceput[0]));
    if (i > 500):
        print("Change: " + str(ampriceput[0] - preval[0]));
    if (i > 1000): 
        print("Ratio: " + str(prechange / (ampriceput[0] - preval[0])));
    prechange = ampriceput[0] - preval[0]
    preval = ampriceput
    print("\n");

    
# test input data from table 3.3.1
# (100, 100, 1, 0.02, 0.3, 1, i)

# data input from assignment
# (100, 100, 0.75, 0.03, 0.35, 1, i)


European call options: 

Exact value from blsprice is 13.0571 

delta = T/500
Value: 13.051177442418886


delta = T/1000
Value: 13.054162112060741
Change: 0.0029846696418545093


delta = T/2000
Value: 13.055654727976728
Change: 0.001492615915987372
Ratio: 1.9996233524551004


delta = T/4000
Value: 13.056401106054865
Change: 0.0007463780781371554
Ratio: 1.9998121055654678


delta = T/8000
Value: 13.056774312619659
Change: 0.00037320656479344905
Ratio: 1.9999060802969473


European put options: 

Exact value from blsprice is 10.8323 

delta = T/500
Value: 10.82630116174909


delta = T/1000
Value: 10.829285831395241
Change: 0.0029846696461515165


delta = T/2000
Value: 10.830778447328267
Change: 0.0014926159330261868
Ratio: 1.999623332507434


delta = T/4000
Value: 10.831524825427628
Change: 0.0007463780993610669
Ratio: 1.9998120715277323


delta = T/8000
Value: 10.831898031968544
Change: 0.0003732065409156604
Ratio: 1.9999062651201984


American put options: 

delta = T/500
Value: 11.026

In [14]:
import numpy as np
import math

### QUESTION 2 ###


# Calculates the option price for American options, but with the option for smoothing 
# the payoff for put options at t=T
# S0 - current stock price 
# K - strike
# T - expiry time
# r - interest rate
# sigma - volatility
# opttype - 0 for a call, otherwise a put 
# Nsteps - number of timesteps
# smooth - 0 for no smoothing at payoff, otherwise apply smoothing at payoff
def amoption(S0, K, T, r, sigma, opttype, Nsteps, smooth): 
    delt = T/Nsteps;
    
    # tree parameters
    u = np.exp(sigma * np.sqrt(delt) ); 
    d = 1./u;
    a = np.exp( r*delt );
    p = (a - d)/(u - d); 
    
    # payoff at t=T
    W = S0 * d**(np.arange(Nsteps,-1,-1)) * u**(np.arange(Nsteps+1))
    
    # W is column vector of length Nsteps+1
    if opttype == 0:
        W = np.maximum(W - K, 0); 
    else:  
        # dont smooth payoff at t=T
        if (smooth == 0):
            W = np.maximum(K - W, 0); 
        # smooth the payoff at t=T
        else: 
            for i in np.arange(1,Nsteps+1):
                if W[i]*d > K:
                    W[i] = 0;
                elif W[i]*u < K:
                    W[i] = K - W[i] * ( (u-d) / (2 * sigma * np.sqrt(delt) ));
                else:
                    W[i] = (1 / (2 * sigma * np.sqrt(delt))) * (
                        (K * (np.log(K / W[i]) + sigma * np.sqrt(delt))) - 
                        (W[i] * ((K / W[i]) - d)))
    
        
    # backward recursion
    for i in np.arange(Nsteps,0,-1):
        S = S0 * d**(np.arange(i-1,-1,-1)) * u**(np.arange(i));     
        W = np.maximum(np.exp(-r*delt)*( p*W[1:i+1] + (1-p)*W[0:i] ), 
                       np.maximum( opttype*(K - S) + (1-opttype)*(S - K) ,0 ));
    
    return W



# Repeat convergence test

teststeps = [500, 1000, 2000, 4000, 8000]
preval = [0]
prechange = 0;

print("American put options without smoothing: \n");

# no smoothing
for i in teststeps:
    ampriceput =  amoption(91, 100, 0.75, 0.03, 0.35, 1, i, 0);
                             
    print("delta = T/" + str(i));
    print("Value: " + str(ampriceput[0]));
    if (i > 500):
        print("Change: " + str(ampriceput[0] - preval[0]));
    if (i > 1000): 
        print("Ratio: " + str(prechange / (ampriceput[0] - preval[0])));
    prechange = ampriceput[0] - preval[0]
    preval = ampriceput
    print("\n");
    
print("American put options with smoothing: \n");

# with smoothing
for i in teststeps:
    ampriceput =  amoption(91, 100, 0.75, 0.03, 0.35, 1, i, 1);
                             
    print("delta = T/" + str(i));
    print("Value: " + str(ampriceput[0]));
    if (i > 500):
        print("Change: " + str(ampriceput[0] - preval[0]));
    if (i > 1000): 
        print("Ratio: " + str(prechange / (ampriceput[0] - preval[0])));
    prechange = ampriceput[0] - preval[0]
    preval = ampriceput
    print("\n");


# test input data from table 3.3.1
# (100, 100, 1, 0.02, 0.3, 1, i, 1)

# test input data from table 3.4.1
# (92, 100, 1, 0.02, 0.3, 1, i, 1)

# data input from assignment
# (91, 100, 0.75, 0.03, 0.35, 1, i, 1)



American put options without smoothing: 

delta = T/500
Value: 15.401609397899962


delta = T/1000
Value: 15.395648822679982
Change: -0.005960575219980058


delta = T/2000
Value: 15.395636532606384
Change: -1.2290073598819617e-05
Ratio: 484.9910110019628


delta = T/4000
Value: 15.396312338644849
Change: 0.0006758060384655096
Ratio: -0.01818580021380921


delta = T/8000
Value: 15.396111510716501
Change: -0.00020082792834763552
Ratio: -3.3650998843929782


American put options with smoothing: 

delta = T/500
Value: 15.401576885493197


delta = T/1000
Value: 15.398765698342569
Change: -0.002811187150628669


delta = T/2000
Value: 15.397474585569027
Change: -0.0012911127735417693
Ratio: 2.177336641877568


delta = T/4000
Value: 15.396822335287718
Change: -0.000652250281309108
Ratio: 1.979474460249252


delta = T/8000
Value: 15.396490458605053
Change: -0.00033187668266521086
Ratio: 1.9653392822631117




Based on the American put type without smoothing, the changes between the values of different delta values are either increasing/decreasing and it looks that the data is not converging since the ratio does not look it is converging at a value.

But with smoothing, the changes are decreasing as the timesteps increase and the ratio grows smaller as well which suggests it will converge at a point.